In [15]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold, cross_val_score


In [9]:
# load resampled dataset 

resampled = pd.read_csv('../data/processed/oversampled.csv')
resampled = resampled.drop(columns='Unnamed: 0')

In [10]:
resampled

,entropy,clicks_count,durations,click_frequency,speed_cv,hover_frequency,avg_hover_duration,acceleration_mean,acceleration_std,acceleration_skewness,...,day_Thursday,day_Tuesday,day_Wednesday,time_afternoon,time_evening,time_morning,time_night,time_wee_hours,hashed_url,label
0,3.121928,10,1606000878926,6.226647e-12,1.292750,3.265045,10044.058824,-97.987654,4076.554706,0.137302,...,0,897,0,0,0,897,0,0,47567188162,human
1,3.221097,19,677103,2.806072e-05,0.509696,0.965173,1951.275000,-3.079521,1211.355072,-0.234733,...,0,0,0,0,401,0,0,0,579555515964,advanced_bot
2,2.895424,17,1606000279969,1.058530e-11,0.517826,0.218432,966.923077,-94.009643,2813.920800,-0.017841,...,0,0,316,0,0,0,316,0,557692383271,advanced_bot
3,2.446439,10,141246,7.079847e-05,0.232604,3.927866,866.327044,-2.978588,616.098440,0.029983,...,0,0,109,109,0,0,0,0,664568366327,moderate_bot
4,3.452820,16,460995,3.470753e-05,0.508165,0.895245,2000.883117,-0.205002,1173.942929,-0.054982,...,0,0,0,0,283,0,0,0,1052243837978,advanced_bot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,3.068451,18,1606000146548,1.123798e-11,0.300744,3.959081,403.973477,-14.169503,553.412508,0.338664,...,0,0,184,0,0,184,0,0,1076285962431,moderate_bot
1496,2.563088,18,1606000128880,1.123083e-11,0.306683,3.908388,414.696321,-22.525232,566.608717,0.494303,...,0,0,169,0,0,169,0,0,683800219037,moderate_bot
1497,2.994324,19,1606000130335,1.238889e-11,0.297990,4.006105,397.818866,-13.048029,635.343817,1.306435,...,0,0,173,0,0,173,0,0,1076285962431,moderate_bot
1498,2.168677,13,1606000083971,8.525991e-12,0.327979,3.957692,421.744266,-18.707041,620.696803,2.140838,...,0,0,117,0,0,117,0,0,683800219037,moderate_bot


Get target column

In [11]:
X = resampled[[column for column in list(resampled.columns) if column != 'label']]
y = resampled[['label']]

In [12]:
# Split dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features (important for KNN)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize KNN classifier with k=3
knn = KNeighborsClassifier(n_neighbors=3)

# Train the model
knn.fit(X_train, y_train)

# Make predictions
y_pred = knn.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'KNN Accuracy: {accuracy:.2f}')

KNN Accuracy: 1.00


/Users/linyan/anaconda3/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


#### With CV

In [16]:
# Define KNN model with preprocessing pipeline
# StandardScaler is usually helpful for KNN
knn = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=3))

# K-Fold setup
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Evaluate model using accuracy (you can use f1, recall, etc. instead)
scores = cross_val_score(knn, X, y, cv=kfold, scoring='accuracy')

# Print results
print(f"Fold Accuracies: {scores}")
print(f"Mean Accuracy: {np.mean(scores):.4f} ± {np.std(scores):.4f}")

Fold Accuracies: [1.         1.         0.99666667 0.99666667 1.        ]
Mean Accuracy: 0.9987 ± 0.0016


/Users/linyan/anaconda3/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/linyan/anaconda3/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/linyan/anaconda3/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/linyan/anaconda3/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected

The dataset is not particularly unbalanced, so there is no need for balancing techniques.

In [14]:
print(len(resampled[resampled['label']== 'human']))
print(len(resampled[resampled['label']== 'moderate_bot']))
print(len(resampled[resampled['label']== 'advanced_bot']))

500
500
500
